In [1]:
import pandas as pd
import pickle
import seaborn as sns
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split 
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from matplotlib import pyplot as plt
from sklearn.model_selection import cross_val_score, KFold, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss, roc_auc_score, recall_score, precision_score, accuracy_score, plot_roc_curve, plot_confusion_matrix, roc_curve, confusion_matrix
import itertools
from tensorflow.keras.initializers import Constant, TruncatedNormal
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.preprocessing import LabelEncoder
from numpy import mean, absolute

In [2]:
metrics = pd.DataFrame(columns=['model', 'group', 'subgroup', 'Sentence', 'Accuracy', 'Precision', 'Recall', 'AUC', 'FPR'])

In [10]:
def get_metrics(clf,X,y,cv,pred):
    #with cv
    # scores_a = cross_val_score(clf, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
    # a = mean(absolute(scores_a))
    # scores_p = cross_val_score(clf, X, y, scoring='precision', cv=cv, n_jobs=-1)
    # p = mean(absolute(scores_p))
    # scores_r = cross_val_score(clf, X, y, scoring='recall', cv=cv, n_jobs=-1)
    # r = mean(absolute(scores_r))
    # scores_auc = cross_val_score(clf, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
    # roc_auc = mean(absolute(scores_auc))

    # #without cv
    a = accuracy_score(y,pred)
    p = precision_score(y,pred)
    r = recall_score(y,pred)
    roc_auc = roc_auc_score(y,pred)
    tn, fp, fn, tp = confusion_matrix(y, pred).ravel()
    fpr = fp/(fp+tn)
    fpr=0
    return a,p,r,roc_auc,fpr

Check amount of training data in evaluation data

In [ ]:
# n = [2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60]
# feature_cols = ['Erstloesung','Schussel','Erfolg', 'Schwierigkeit', 'ist_Schulzeit', 'MehrfachFalsch', 'vorher_abgebrochen','Fehler', 'Klassenstufe', 'Jahredabei', 'AnzahlAufgaben', 'Sex__m', 'Sex__w', 'Testposition__pruefung', 'Testposition__training','Testposition__version', 'Art__GK', 'Art__GR', 'Art__GZ', 'Art__K', 'Art__LB','UserAttribut', 'OrderNumber', 'steps']
# fails = pd.DataFrame(columns=['n','group', 'group_len', 'removed_train', 'exploited_rows'])

# for i in n:
#     #build models
#     path='matrices_allsessions/matrix'+str(i)+'.pkl'
#     infile = open(path,'rb')
#     df = pickle.load(infile)
#     infile.close()
#     df=df.reset_index()
#     X = df[feature_cols]
#     y = df.y
#     y= y.astype('int')
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

#     group = ['abiEltern', 'abiEltern', 'gender', 'gender', 'erstsprache', 'erstsprache', 'buecher', 'buecher', 'bedeutungNoten', 'bedeutungNoten']
#     subgroup = ['abi', 'keinAbi', 'boys', 'girls', 'deutsch', 'migration', 'buch0', 'buch1', 'bedeutung0','bedeutung1']
#     matrice = ['matrices_forte_abi', 'matrices_forte_keinAbi','matrices_forte_boys','matrices_forte_girls', 'matrices_forte_deutsch', 'matrices_forte_migration', 'matrices_buecher_0', 'matrices_buecher_1', 'matrices_bedeutung0', 'matrices_bedeutung1']

#     for (group, subgroup, matrix) in zip(group, subgroup, matrice):
#         path= matrix+'/matrix'+str(i)+'.pkl'
#         infile = open(path,'rb')
#         df = pickle.load(infile)
#         infile.close()
#         df=df.reset_index()
#         X = df[feature_cols]

#         X_new = X.loc[~X.set_index(list(X.columns)).index.isin(X_train.set_index(list(X_train.columns)).index)]
#         fails = fails.append({'n':i,'group':matrix, 'group_len':len(X), 'removed_train':len(X_new), 'exploited_rows':(len(X)-len(X_new))}, ignore_index=True)

# fails.to_excel('fails.xlsx')


Decision Tree without Training Data

In [ ]:
n = [2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60]
feature_cols = ['Erstloesung','Schussel','Erfolg', 'Schwierigkeit', 'ist_Schulzeit', 'MehrfachFalsch', 'vorher_abgebrochen','Fehler', 'Klassenstufe', 'Jahredabei', 'AnzahlAufgaben', 'Sex__m', 'Sex__w', 'Testposition__pruefung', 'Testposition__training','Testposition__version', 'Art__GK', 'Art__GR', 'Art__GZ', 'Art__K', 'Art__LB','UserAttribut', 'OrderNumber', 'steps']
feature_cols_y = ['Erstloesung','Schussel','Erfolg', 'Schwierigkeit', 'ist_Schulzeit', 'MehrfachFalsch', 'vorher_abgebrochen','Fehler', 'Klassenstufe', 'Jahredabei', 'AnzahlAufgaben', 'Sex__m', 'Sex__w', 'Testposition__pruefung', 'Testposition__training','Testposition__version', 'Art__GK', 'Art__GR', 'Art__GZ', 'Art__K', 'Art__LB','UserAttribut', 'OrderNumber', 'steps','y']

for i in n:
    #build models
    path='matrices_allsessions/matrix'+str(i)+'.pkl'
    infile = open(path,'rb')
    df = pickle.load(infile)
    infile.close()
    df=df.reset_index()
    X = df[feature_cols]
    y = df.y
    y= y.astype('int')
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)
    train = X_train.assign(y = y_train)

    k = 5
    cv = KFold(n_splits=k, random_state=None)

    #DTE
    clf = DecisionTreeClassifier(criterion="entropy", max_depth=5)
    clf = clf.fit(X_train,y_train)
    pred = clf.predict(X_test)

    a,p,r,roc_auc,fpr = get_metrics(clf,X_test,y_test,cv,pred)

    metrics = metrics.append({'model':'DTE','group':'all','subgroup':'all','Sentence': i, 'Accuracy':a,'Precision': p, 'Recall':r, 'AUC':roc_auc, 'FPR':fpr}, ignore_index=True)

    group = ['abiEltern', 'abiEltern', 'gender', 'gender', 'erstsprache', 'erstsprache', 'buecher', 'buecher', 'bedeutungNoten', 'bedeutungNoten']
    subgroup = ['abi', 'keinAbi', 'boys', 'girls', 'deutsch', 'migration', 'buch0', 'buch1', 'bedeutung0','bedeutung1']
    matrice = ['matrices_forte_abi', 'matrices_forte_keinAbi','matrices_forte_boys','matrices_forte_girls', 'matrices_forte_deutsch', 'matrices_forte_migration', 'matrices_buecher_0', 'matrices_buecher_1', 'matrices_bedeutung0', 'matrices_bedeutung1']

    for (group, subgroup, matrix) in zip(group, subgroup, matrice):
        path= matrix+'/matrix'+str(i)+'.pkl'
        infile = open(path,'rb')
        df = pickle.load(infile)
        infile.close()
        df=df.reset_index()
        dataset = df[feature_cols_y]
        X_new = dataset.loc[~dataset.set_index(list(dataset.columns)).index.isin(train.set_index(list(train.columns)).index)]

        X = X_new[feature_cols]
        y = X_new.y
        y= y.astype('int')
        pred = clf.predict(X)
        
        a,p,r,roc_auc,fpr = get_metrics(clf,X,y,cv,pred)
        metrics = metrics.append({'model':'DTE','group':group,'subgroup':subgroup,'Sentence': i, 'Accuracy':a,'Precision': p, 'Recall':r, 'AUC':roc_auc, 'FPR':fpr}, ignore_index=True)

#metrics.to_excel('dte_metrics_withoutTraining.xlsx')


Decision Tree Classifier, builds model for each t, evaluates for each subgroup and saves the metrics in list

In [11]:
n = [2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60]
feature_cols = ['Erstloesung','Schussel','Erfolg', 'Schwierigkeit', 'ist_Schulzeit', 'MehrfachFalsch', 'vorher_abgebrochen','Fehler', 'Klassenstufe', 'Jahredabei', 'AnzahlAufgaben', 'Sex__m', 'Sex__w', 'Testposition__pruefung', 'Testposition__training','Testposition__version', 'Art__GK', 'Art__GR', 'Art__GZ', 'Art__K', 'Art__LB','UserAttribut', 'OrderNumber', 'steps']

for i in n:
    #build models
    path='../01_keep_it_up/matrices_allsessions/matrix'+str(i)+'.pkl'
    infile = open(path,'rb')
    df = pickle.load(infile)
    infile.close()
    df=df.reset_index()
    X = df[feature_cols]
    y = df.y
    y= y.astype('int')
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)
    k = 5
    cv = KFold(n_splits=k, random_state=None)

    #DTE
    clf = DecisionTreeClassifier(criterion="entropy", max_depth=5)
    clf = clf.fit(X_train,y_train)
    pred = clf.predict(X_test)

    a,p,r,roc_auc,fpr = get_metrics(clf,X_test,y_test,cv,pred)

    metrics = metrics.append({'model':'DTE','group':'all','subgroup':'all','Sentence': i, 'Accuracy':a,'Precision': p, 'Recall':r, 'AUC':roc_auc, 'FPR':fpr}, ignore_index=True)

    group = ['abiEltern', 'abiEltern', 'gender', 'gender', 'erstsprache', 'erstsprache', 'buecher', 'buecher', 'bedeutungNoten', 'bedeutungNoten']
    subgroup = ['abi', 'keinAbi', 'boys', 'girls', 'deutsch', 'migration', 'buch0', 'buch1', 'bedeutung0','bedeutung1']
    matrice = ['matrices_forte_abi', 'matrices_forte_keinAbi','matrices_forte_boys','matrices_forte_girls', 'matrices_forte_deutsch', 'matrices_forte_migration', 'matrices_forte_buecher_0', 'matrices_forte_buecher_1', 'matrices_bedeutung0', 'matrices_bedeutung1']

    for (group, subgroup, matrix) in zip(group, subgroup, matrice):
        path= '../01_keep_it_up/'+matrix+'/matrix'+str(i)+'.pkl'
        infile = open(path,'rb')
        df = pickle.load(infile)
        infile.close()
        df=df.reset_index()
        X = df[feature_cols]
        y = df.y
        y= y.astype('int')
        pred = clf.predict(X)
        
        a,p,r,roc_auc,fpr = get_metrics(clf,X,y,cv,pred)
        metrics = metrics.append({'model':'DTE','group':group,'subgroup':subgroup,'Sentence': i, 'Accuracy':a,'Precision': p, 'Recall':r, 'AUC':roc_auc, 'FPR':fpr}, ignore_index=True)

C:\Users\Nathalie\AppData\Local\Temp\ipykernel_24696\2367966947.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics = metrics.append({'model':'DTE','group':'all','subgroup':'all','Sentence': i, 'Accuracy':a,'Precision': p, 'Recall':r, 'AUC':roc_auc, 'FPR':fpr}, ignore_index=True)
C:\Users\Nathalie\AppData\Local\Temp\ipykernel_24696\2367966947.py:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics = metrics.append({'model':'DTE','group':group,'subgroup':subgroup,'Sentence': i, 'Accuracy':a,'Precision': p, 'Recall':r, 'AUC':roc_auc, 'FPR':fpr}, ignore_index=True)
C:\Users\Nathalie\AppData\Local\Temp\ipykernel_24696\2367966947.py:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics = metrics.append({'model'

In [12]:
metrics

,model,group,subgroup,Sentence,Accuracy,Precision,Recall,AUC,FPR
0,DTE,all,all,2,0.75918,0.759524,0.861349,0.816978,0
1,DTE,all,all,2,0.75918,0.759524,0.861349,0.816978,0
2,DTE,abiEltern,abi,2,0.896081,0.902057,0.98939,0.794881,0
3,DTE,abiEltern,keinAbi,2,0.88209,0.892156,0.983876,0.758498,0
4,DTE,gender,boys,2,0.869919,0.874194,0.988619,0.801803,0
...,...,...,...,...,...,...,...,...,...
1301,DTE,erstsprache,migration,60,0.921225,0.977778,0.93617,0.835732,0
1302,DTE,buecher,buch0,60,0.9,0.965517,0.918033,0.847905,0
1303,DTE,buecher,buch1,60,0.935961,0.986523,0.945736,0.841289,0
1304,DTE,bedeutungNoten,bedeutung0,60,1.0,1.0,1.0,1.0,0


In [9]:
metrics

,model,group,subgroup,Sentence,Accuracy,Precision,Recall,AUC,FPR
0,DTE,all,all,2,0.75918,0.759524,0.861349,0.816978,0
1,DTE,all,all,2,0.75918,0.759524,0.861349,0.816978,0
2,DTE,abiEltern,abi,2,0.896081,0.902057,0.98939,0.794881,0
3,DTE,abiEltern,keinAbi,2,0.88209,0.892156,0.983876,0.758498,0
4,DTE,gender,boys,2,0.869919,0.874194,0.988619,0.801803,0
...,...,...,...,...,...,...,...,...,...
652,DTE,erstsprache,migration,60,0.912566,0.94011,0.973911,0.627804,0
653,DTE,buecher,buch0,60,0.785714,0.924206,0.827213,NaN,0
654,DTE,buecher,buch1,60,0.94586,0.967298,0.989809,0.708782,0
655,DTE,bedeutungNoten,bedeutung0,60,0.938095,0.885714,0.966667,NaN,0


KNN

In [ ]:
n =  [2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60]
feature_cols = ['Erstloesung','Schussel','Erfolg', 'Schwierigkeit', 'ist_Schulzeit', 'MehrfachFalsch', 'vorher_abgebrochen','Fehler', 'Klassenstufe', 'Jahredabei', 'AnzahlAufgaben', 'Sex__m', 'Sex__w', 'Testposition__pruefung', 'Testposition__training','Testposition__version', 'Art__GK', 'Art__GR', 'Art__GZ', 'Art__K', 'Art__LB','UserAttribut', 'OrderNumber', 'steps']

for i in n:
    #build models
    path='matrices_allsessions/matrix'+str(i)+'.pkl'
    infile = open(path,'rb')
    df = pickle.load(infile)
    infile.close()
    df=df.reset_index()
    X = df[feature_cols]
    y = df.y
    y= y.astype('int')
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)
    k = 5
    cv = KFold(n_splits=k, random_state=None)

    #knn
    knn = KNeighborsClassifier(n_neighbors=2)
    knn = knn.fit(X_train, y_train)#clf

    pred = knn.predict(X_test)#cf
    a,p,r,roc_auc,fpr = get_metrics(knn,X_test,y_test,cv,pred)
    metrics = metrics.append({'model':'KNN','group':'all','subgroup':'all','Sentence': i, 'Accuracy':a,'Precision': p, 'Recall':r, 'AUC':roc_auc, 'FPR':fpr}, ignore_index=True)

    group = ['abiEltern', 'abiEltern', 'gender', 'gender', 'erstsprache', 'erstsprache', 'buecher', 'buecher', 'bedeutungNoten', 'bedeutungNoten']
    subgroup = ['abi', 'keinAbi', 'boys', 'girls', 'deutsch', 'migration', 'buch0', 'buch1', 'bedeutung0','bedeutung1']
    matrice = ['matrices_forte_abi', 'matrices_forte_keinAbi','matrices_forte_boys','matrices_forte_girls', 'matrices_forte_deutsch', 'matrices_forte_migration', 'matrices_buecher_0', 'matrices_buecher_1', 'matrices_bedeutung0', 'matrices_bedeutung1']
    for (group, subgroup, matrix) in zip(group, subgroup, matrice):
        print(subgroup)
        path= matrix+'/matrix'+str(i)+'.pkl'
        infile = open(path,'rb')
        df = pickle.load(infile)
        infile.close()
        df=df.reset_index()
        X = df[feature_cols]
        y = df.y
        y= y.astype('int')
        pred = knn.predict(X)#clf

        a,p,r,roc_auc,fpr = get_metrics(knn,X,y,cv,pred)#clf
        metrics = metrics.append({'model':'KNN','group':group,'subgroup':subgroup,'Sentence': i, 'Accuracy':a,'Precision': p, 'Recall':r, 'AUC':roc_auc, 'FPR':fpr}, ignore_index=True)

Deep NN

In [ ]:
# baseline model
def build_model():
	# create model
	model = Sequential()
	model.add(Dense(24, input_dim=24, activation='relu'))
	model.add(Dense(48, activation='relu'))
	model.add(Dense(24, activation='relu'))
	model.add(Dense(12, activation='relu'))
	model.add(Dense(1, activation='sigmoid'))
		
	return model


def get_dn_metrics(model, X,y):
    yhat_probs = model.predict(X, verbose=0)
    yhat_classes =  (model.predict(X) > 0.5).astype("int32")
    # reduce to 1d array
    yhat_probs = yhat_probs[:, 0]
    yhat_classes = yhat_classes[:, 0]
    a = accuracy_score(y, yhat_classes)
    p = precision_score(y, yhat_classes)
    r = recall_score(y, yhat_classes)
    roc_auc = roc_auc_score(y, yhat_probs)
    tn, fp, fn, tp = confusion_matrix(y, yhat_classes).ravel()
    fpr = fp/(fp+tn)

    return a,p,r,roc_auc,fpr

n =  [2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60]
feature_cols = ['Erstloesung','Schussel','Erfolg', 'Schwierigkeit', 'ist_Schulzeit', 'MehrfachFalsch', 'vorher_abgebrochen','Fehler', 'Klassenstufe', 'Jahredabei', 'AnzahlAufgaben', 'Sex__m', 'Sex__w', 'Testposition__pruefung', 'Testposition__training','Testposition__version', 'Art__GK', 'Art__GR', 'Art__GZ', 'Art__K', 'Art__LB','UserAttribut', 'OrderNumber', 'steps']

for i in n:
    path='matrices_allsessions/matrix'+str(i)+'.pkl'
    infile = open(path,'rb')
    df = pickle.load(infile)
    infile.close()
    df=df.reset_index()

    y_len = len(feature_cols)
    X = df[feature_cols].astype(float)
    y = df.y
    y= y.astype('int')
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

    model = build_model()

    model.compile(
        loss="binary_crossentropy",
        optimizer='Adam',
        metrics=["accuracy"]
    )

    model.fit(
        x=X_train,
        y=y_train,
        epochs=10,
        batch_size=128,
        verbose=0,
        validation_data=(X_test, y_test)
    )

    scores = model.evaluate(
        x=X_test,
        y=y_test,
        verbose=0
    )

    a,p,r,roc_auc,fpr= get_dn_metrics(model, X_test,y_test)
    metrics = metrics.append({'model':'DL','group':'all','subgroup':'all','Sentence': i, 'Accuracy':a,'Precision': p, 'Recall':r, 'AUC':roc_auc, 'FPR':fpr}, ignore_index=True)

    group = ['abiEltern', 'abiEltern', 'gender', 'gender', 'erstsprache', 'erstsprache', 'buecher', 'buecher', 'bedeutungNoten', 'bedeutungNoten']
    subgroup = ['abi', 'keinAbi', 'boys', 'girls', 'deutsch', 'migration', 'buch0', 'buch1', 'bedeutung0','bedeutung1']
    matrice = ['matrices_forte_abi', 'matrices_forte_keinAbi','matrices_forte_boys','matrices_forte_girls', 'matrices_forte_deutsch', 'matrices_forte_migration', 'matrices_buecher_0', 'matrices_buecher_1', 'matrices_bedeutung0', 'matrices_bedeutung1']
    
    for (group, subgroup, matrice) in zip(group, subgroup, matrice):
        print(subgroup)
        path=matrice+'/matrix'+str(i)+'.pkl' 
        infile = open(path,'rb')
        df = pickle.load(infile)
        infile.close()
        df=df.reset_index()
        y_len = len(feature_cols)
        X = df[feature_cols].astype(float)
        y = df.y
        y= y.astype('int')

        a,p,r,roc_auc,fpr= get_dn_metrics(model, X,y)

        metrics = metrics.append({'model':'DL','group':group,'subgroup':subgroup,'Sentence': i, 'Accuracy':a,'Precision': p, 'Recall':r, 'AUC':roc_auc, 'FPR':fpr}, ignore_index=True)

In [ ]:
#print(metrics)
metrics.to_excel('all_metrics.xlsx')
metrics.to_pickle('all_metrics.pkl')
